In [1]:
import os
import warnings
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing import image

In [2]:
# Define the paths to the dataset
train_dir = r'C:\Users\Sathish\plant disease\train'  
val_dir = r'C:\Users\Sathish\plant disease\valid'  

In [3]:
# Data augmentation for training
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    zoom_range=0.2,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    horizontal_flip=True
)

In [4]:
# For validation, just rescale
val_datagen = ImageDataGenerator(rescale=1./255)

In [5]:
# Create generators for training and validation datasets
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'
)

Found 9281 images belonging to 5 classes.


In [6]:
val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'
)

Found 2319 images belonging to 5 classes.


In [7]:
# Check the number of classes
num_classes = len(train_generator.class_indices)
print("Number of classes:", num_classes)

Number of classes: 5


In [8]:
# Define the model architecture
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(num_classes, activation='softmax')  # Output layer matches number of classes
])

C:\Users\Sathish\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [9]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [14]:
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10,  # Adjust the number of epochs as needed
    steps_per_epoch=train_generator.samples // 32,
    validation_steps=val_generator.samples // 32
)

Epoch 1/10


C:\Users\Sathish\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


290/290 ━━━━━━━━━━━━━━━━━━━━ 182s 616ms/step - accuracy: 0.4319 - loss: 1.2746 - val_accuracy: 0.4557 - val_loss: 2.2866
Epoch 2/10
290/290 ━━━━━━━━━━━━━━━━━━━━ 0s 291us/step - accuracy: 0.6250 - loss: 1.0794 - val_accuracy: 0.8000 - val_loss: 1.0150
Epoch 3/10


C:\Users\Sathish\anaconda3\Lib\contextlib.py:155: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


290/290 ━━━━━━━━━━━━━━━━━━━━ 204s 699ms/step - accuracy: 0.7454 - loss: 0.6622 - val_accuracy: 0.7279 - val_loss: 0.9995
Epoch 4/10
290/290 ━━━━━━━━━━━━━━━━━━━━ 0s 337us/step - accuracy: 0.9688 - loss: 0.2113 - val_accuracy: 0.8000 - val_loss: 0.9589
Epoch 5/10
290/290 ━━━━━━━━━━━━━━━━━━━━ 194s 660ms/step - accuracy: 0.8561 - loss: 0.3940 - val_accuracy: 0.8264 - val_loss: 0.6371
Epoch 6/10
290/290 ━━━━━━━━━━━━━━━━━━━━ 0s 261us/step - accuracy: 0.8438 - loss: 0.3145 - val_accuracy: 0.7333 - val_loss: 1.2698
Epoch 7/10
290/290 ━━━━━━━━━━━━━━━━━━━━ 144s 492ms/step - accuracy: 0.8807 - loss: 0.3275 - val_accuracy: 0.8585 - val_loss: 0.4724
Epoch 8/10
290/290 ━━━━━━━━━━━━━━━━━━━━ 0s 226us/step - accuracy: 0.8750 - loss: 0.3446 - val_accuracy: 0.8667 - val_loss: 0.1655
Epoch 9/10
290/290 ━━━━━━━━━━━━━━━━━━━━ 141s 480ms/step - accuracy: 0.9038 - loss: 0.2657 - val_accuracy: 0.7630 - val_loss: 1.3516
Epoch 10/10
290/290 ━━━━━━━━━━━━━━━━━━━━ 0s 220us/step - accuracy: 0.8125 - loss: 0.3640 - va

In [19]:
# Save the model (optional)
model.save('tomato_disease_classifier.h5')

In [21]:
# dictionary for disease to pesticide recommendations
pesticide_recommendations = {
    'Tomato___Bacterial_spot': 'Use Copper-based fungicides',
    'Tomato___Early_blight': 'Use Chlorothalonil or Azoxystrobin',
    'Tomato___healthy': 'No pesticides needed',
    'Tomato___Late_blight': 'Use Mancozeb or Ridomil',
    'Tomato___Leaf_Mold': 'Use Potassium bicarbonate or Neem oil'
}

In [22]:
# Feature-based pesticide recommendation
def get_pesticide_recommendation(features):
    if features.get('spotting', 0) > 0.5 and features.get('blight_level', 0) < 0.3:
        return "Use Copper-based fungicides"
    elif features.get('blight_level', 0) > 0.7:
        return "Use Chlorothalonil or Azoxystrobin"
    elif features.get('leaf_health', 0) > 0.9:
        return "No pesticides needed"
    elif features.get('late_blight_indicator', 0) > 0.6:
        return "Use Mancozeb or Ridomil"
    elif features.get('mold_presence', 0) > 0.5:
        return "Use Potassium bicarbonate or Neem oil"
    else:
        return "No specific recommendation available"


In [23]:
# Hypothetical feature extraction function
def extract_features(img_array):
    # Placeholder for feature extraction logic, replace with actual feature extraction as needed
    return {
        'spotting': np.random.rand(),         # Replace with real extraction logic
        'blight_level': np.random.rand(),
        'leaf_health': np.random.rand(),
        'late_blight_indicator': np.random.rand(),
        'mold_presence': np.random.rand()
    }

In [24]:
def recommend_pesticide(disease):
    return pesticide_recommendations.get(disease, "No recommendation available.")

def predict_and_recommend(image_path):
    # Load and preprocess the image
    img = image.load_img(image_path, target_size=(150, 150))
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    
    # Make predictions
    prediction = model.predict(img_array)
    predicted_class = np.argmax(prediction, axis=1)
    
    # Print prediction probabilities for debugging
    print("Prediction probabilities:", prediction)
    
    # Map predicted class to disease name
    disease_name = list(train_generator.class_indices.keys())[predicted_class[0]]
    
    # Get pesticide recommendation
    pesticide = recommend_pesticide(disease_name)
    
    return disease_name, pesticide


In [25]:
# Example usage
image_path = r'C:\Users\Sathish\plant disease\valid\Tomato___Late_blight\1e5ba644-efeb-4bd3-b878-a0606cf8a992___RS_Late.B 6272_flipLR.JPG'  
disease, pesticide = predict_and_recommend(image_path)
print(f'Detected Disease: {disease}\nRecommended Pesticide: {pesticide}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step
Prediction probabilities: [[2.9516203e-12 1.1513952e-01 8.5376352e-01 1.9482749e-10 3.1096995e-02]]
Detected Disease: Tomato___Late_blight
Recommended Pesticide: Use Mancozeb or Ridomil
